In [1]:
import os

from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

from agent_framework import AgentRunResponse,ChatAgent,HostedFileSearchTool,HostedVectorStoreContent
from agent_framework.azure import AzureAIAgentClient

In [2]:
load_dotenv()

True

In [3]:
async def create_vector_store(client: AzureAIAgentClient) -> tuple[str, HostedVectorStoreContent]:
    """Create a vector store with sample documents."""
    file_path = '../../files/demo.md'
    file = await client.project_client.agents.files.upload_and_poll(file_path=file_path, purpose="assistants")
    print(f"Uploaded file, file ID: {file.id}")


    vector_store = await client.project_client.agents.vector_stores.create_and_poll(file_ids=[file.id], name="graph_knowledge_base")

    print(f"Created vector store, ID: {vector_store.id}")


    return file.id, HostedVectorStoreContent(vector_store_id=vector_store.id)

In [4]:
async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        file_id, vector_store = await create_vector_store(chat_client)

        file_search = HostedFileSearchTool(inputs=vector_store)
        
        agent = chat_client.create_agent(
            name="PythonRAGDemo",
            instructions="""You are an AI assistant that helps people find information in a set of documents. Use the File Search tool to look up relevant information from the files when needed to answer user questions. If you don't know the answer, just say you don't know. Do not make up answers.
                """,
            tools=[file_search],  # Tools available to the agent
            tool_choice = "auto",  # Let the agent decide when to use tools
        )
                

        print("Agent created. You can now ask questions about the uploaded document.")

        query = "What's GraphRAG?"
        async for chunk in agent.run_stream(query, tool_resources={"file_search": {"vector_store_ids": [vector_store.vector_store_id]}}):
                
            if chunk.text:
                print(chunk.text, end="", flush=True)

Uploaded file, file ID: assistant-VmANqc1CMnLJsAEh2qatnc
Created vector store, ID: vs_uAsA1TJlNbYWjnYYjxO09rvn
Agent created. You can now ask questions about the uploaded document.
GraphRAG is an AI-based content interpretation and search system that uses large language models (LLMs) to parse data and create a knowledge graph for answering user questions about a provided private dataset. It excels at connecting information across large volumes of data to answer complex questions that span many documents or address thematic aspects, which are difficult for typical keyword or vector-based search systems to handle.

GraphRAG is designed to support critical information discovery and analysis use cases involving noisy data, misinformation, or abstract questions. It is meant for users trained in responsible analytic approaches who can apply domain expertise to verify and augment the answers produced. It works with domain-specific text corpora and does not collect user data itself.

The syste